# Transportation Problem

## Business Problem
A company operates multiple warehouses and serves several customer regions.
Each warehouse has a limited supply capacity, and each customer region has a
known demand. Transportation costs vary between each warehouse-customer region pairs.

### Goal
Determine the optimal shipment quantities from each warehouse to each region
that **minimize total transportation cost**.

### Constraints
- Each warehouse cannot ship more than its available supply.
- Demand at each customer region must be fully satisfied.
- Shipment quantities must be non-negative.

        
 

## Mathematical Formulation

#### Sets and Parameters
\begin{array}
 \mathcal{}\mathcal{I}&: \text{Set of warehouses (supply nodes)},\enspace i\in\mathcal{I} \\
\mathcal{J} &: \text{Set of customer regions (demand nodes)},\enspace j\in\mathcal{J} \\
s_i &: \text{Capacity of the warehouse }i\\
d_i &: \text{Demand at the customer region }j\\
c_{ij} &: \text{Unit transportation cost from the warehouse } i \text{ to the customer region } j
\end{array}

#### Decision Variable
\begin{equation}
x_{ij} : \text{Quantities shipped from the warehouse } i \text{ to the customer region
}j
\end{equation}

#### Objective Function
\begin{equation}
\min\quad \sum_{i\in\mathcal{I}}\sum_{j\in\mathcal{J}}c_{ij}~x_{ij}
\end{equation}
**Interpretation:**
Minimize the total transportation cost across shipping routes of all warehouse-customer region pairs.

#### Constraints
1. **Supply Capacity Limit** \begin{equation} \sum_{j\in\mathcal{J}}x_{ij}\leq s_i,\enspace\forall~ i\in\mathcal{I}\end{equation} **Interpretation:**
Each warehouse cannot ship more than its available supply capacity.
2. **Demand Satisfaction Constraint** \begin{equation}\sum_{i\in\mathcal{I}}x_{ij}=d_j,\enspace \forall ~j\in\mathcal{J}\end{equation} **Interpretation:**
The demand of each customer region  must be fully satisfied.
4. **Nonnegative Shipment**\begin{equation}x_{ij}\geq 0,\enspace \forall ~i\in\mathcal{I},j\in\mathcal{J}\end{equation}**Interpretation:** Negative shipment quantities are not possible.

## Problem Data and Assumptions

To illustrate the transportation problem clearly and to demonstrate the
model scalability, we consider two data configurations:

1. **Reference Scenario**  
   A small illustrative example with a few warehouses and customer regions, used to
   clearly explain the decision structure and solution behavior.

2. **Large Scale Transportation Scenario**  
   A larger, programmatically generated dataset that demonstrates how the same
   model formulation scales to more realistic problem sizes.

The optimization model formulation remains identical in both cases.


## From Business Problem to Optimization Code

To solve a business decision problem using optimization, the implementation is
structured into three clear stages:

1. **Model Construction**  
   The mathematical structure of the business problem is translated into an
   optimization model using a model-building function. This step defines
   decision variables, the objective function, and constraints, irrespective of the paramters.

2. **Scenario-Specific Data**  
   Problem parameters such as supply, demand, costs and the number of warehouses and customer regions are defined separately
   for each scenario. These parameters may vary depending on the scale and
   requirements of the problem.

3. **Solve and Analyze**  
   The optimization model is solved using an appropriate solver, and the
   resulting decisions are extracted and presented in a readable format to
   support interpretation and analysis.

This separation ensures that the same optimization model can be reused across
different scenarios while keeping the implementation clean, scalable, and
easy to extend.


### Reusable Optimization Model and Utilities

To support clean design and scalability, the optimization logic is implemented
as a set of reusable, scenario-independent functions.

This section defines:
- A model-building function that translates the 
  mathematical structure of the transportation problem into a Pyomo optimization model
- A solver interface that enables the model formulation to be solved using
  different optimization solvers by updating the model with given inputs
- A function to display optimal shipping decisions in a clear,
  tabular format

By separating model formulation, solver execution, and result presentation from
scenario-specific data, the same optimization model can be applied consistently
to both small illustrative examples and larger-scale transportation scenarios.


In [1]:
# Build a Pyomo model for the transportation problem

def build_transportation_model(warehouses, customer_regions, supply_capacity, demand, cost):
    """
    Build and returns a Pyomo transportation model

    Parameter:
        - warehouses
        - customer regions
        - supply capacity of the warehouse
        - demands at the customer regions
        - unit cost of transportation

    Returns: 
        - a pyomo model of the problem
    """
    model = pyo.ConcreteModel()

    # Define the sets
    model.I = pyo.Set(initialize = warehouses)
    model.J = pyo.Set(initialize = customer_regions)

    # Declare the decision variables
    model.x = pyo.Var(model.I, model.J, domain = pyo.NonNegativeReals)

    # Objective function
    def obj_rule(m):
        return sum(cost[i][j] * m.x[i,j] for i in m.I for j in m.J)
        
    model.obj = pyo.Objective(
        rule = obj_rule,
        sense = pyo.minimize
    )

    # Constraint 1: Supply limts
    def supply_rule(m,i):
        return sum(m.x[i,j] for j in m.J) <= supply_capacity[i]
        
    model.supply_constraint = pyo.Constraint(model.I, rule = supply_rule)

    # Constraint 2: Demand Satisfaction
    def demand_rule(m,j):
        return sum(m.x[i,j] for i in m.I) == demand[j]
    
    model.demand_constraint = pyo.Constraint(model.J, rule = demand_rule)
    
    return model



In [2]:
# Solve the built pyomo model of the transportation problem with a specified solver

def solve_transportation_problem(model, solver_name):
    """
    Solve a Pyomo model using the specified solver
    """
    solver = pyo.SolverFactory(solver_name)
    results = solver.solve(model, tee = True)
    return results

In [3]:
# Extract the optimal solutions to a readable table format

import pandas as pd

def print_shipment_route(model, title = "Optimal Shipment Plan", tol=1e-6):
    """
    Prints the optimal shipment plan as a table

     Paramters:
    -----------
    model: Pyomo model 
        Solved transportation problem model with decision variable x[i,j]
    title: str
        Optimal shipment route
    tol: float
        Tolerence for filtering zero shipment
    """
    data = []
    for i in model.I:
        for j in model.J:
            quantity = pyo.value(model.x[i,j])
            if quantity > tol:
                data.append({
                    "Warehouse": i,
                    "Customer Region": j,
                    "Units Shipped": quantity
                })
                
    # Create dataframe
    results_df = pd.DataFrame(data)

    if results_df.empty:
        print("No shipments to display")
        return

    line = "="*len(title)    

    print(f"\n{line}\n{title}\n{line}")
    display(
    results_df
    .style
    .hide(axis="index")
    .format({"Units Shipped": "{:.1f}"})
)



## Transportation Model: Reference Scenario

We first consider a small transportation network consisting of:

- **2 Warehouses**  
- **3 Customer regions**  

Supply, demand, and transportation costs are fixed and chosen to make the
optimal shipping decisions easy to interpret.


In [4]:
## Parameter of the transportation problem (Reference Scenario)

import pyomo.environ as pyo

#Small illustrative dataset
warehouses = ['W1', 'W2']
customer_regions = ['R1', 'R2', 'R3']

supply_capacity = {'W1':100, 'W2':150}
demand = {'R1':80, 'R2':120, 'R3':50}

cost = {
    'W1': {'R1':4, 'R2':6, 'R3':9},
    'W2': {'R1':5, 'R2':4, 'R3':7}
}

In [5]:

# 1. Build the pyomo model
model = build_transportation_model(warehouses, customer_regions, supply_capacity, demand, cost)

# 2. Select the solver
solver_name = 'cbc'

# 3. Solve the optimization problem
results = solve_transportation_problem(model, solver_name)

# 4. Print the results
cbc_optimal = pyo.value(model.obj)
print(f"Total transportation cost =  {cbc_optimal}")
print_shipment_route(model, title = "Optimal Shipment Plan", tol=1e-6)


Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Jan 21 2024 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpc8h2h614.pyomo.lp -stat=1 -solve -solu /tmp/tmpc8h2h614.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2 (-3) rows, 2 (-4) columns and 4 (-8) elements
Statistics for presolved model


Problem has 2 rows, 2 columns (2 with objective) and 4 elements
Column breakdown:
0 of type 0.0->inf, 2 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
2 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 2 (-3) rows, 2 (-4) columns and 4 (-8) elements
0  Obj 1269.8 Primal inf 80.100001 (1) Dual inf 0.99999

Warehouse,Customer Region,Units Shipped
W1,R1,80.0
W1,R3,20.0
W2,R2,120.0
W2,R3,30.0


### Results Interpretation (Reference scenario)

The optimal solution assigns shipments along the lowest-cost routes while fully
satisfying customer demand and respecting warehouse supply limits.

Key observations:
1. Lower-cost shipping routes are preferred whenever capacity allows.
2. All regional demands are met exactly.
3. The linear programming formulation yields meaningful shipment decisions
   without requiring integer constraints.


## Large Scale Transportation Scenario

To assess how the optimization model performs for more realistic problem sizes, we consider a larger transportation problem.

- Warehouse and region indices are created programmatically.
- Supply, demand, and transportation costs are generated using a fixed random
  seed to ensure reproducibility.
- The same optimization model formulation is reused without modification.


In [6]:
# Generating parameters for the Large Scale Transportation Problem

import random
random.seed(42)

num_warehouses = 5
num_customer_regions = 10

warehouses = [f"W{i}" for i in range(1, num_warehouses+1)]
customer_regions = [f"R{j}" for j in range(1, num_customer_regions+1)]

supply_capacity = {i: random.randint(80,200) for i in warehouses}
demand = {j: random.randint(20,60) for j in customer_regions}

cost = {i: {j: random.randint(3,10) for j in customer_regions} for i in warehouses}

# Feasibility check

if sum(supply_capacity.values()) < sum(demand.values()):
    raise ValueError("Total supply is less than the total demand")



In [7]:

# 1. Build the pyomo model
scalable_model = build_transportation_model(warehouses, customer_regions, supply_capacity, demand, cost)

# 2. Select the solver
solver_name = 'cbc'

# 3. Solve the optimization problem
scalable_results = solve_transportation_problem(scalable_model, solver_name)

# 4. Print the results
cbc_optimal = pyo.value(scalable_model.obj)
print(f"Total transportation cost =  {cbc_optimal}")
print_shipment_route(scalable_model, title = "Optimal Shipment Plan", tol=1e-6)


# # Solving the scalable example

# scalable_model = solve_transportation_problem(
#     warehouses, customer_regions, supply_capacity, demand, cost
# )

# print(f"Total transportation cost = {pyo.value(scalable_model.obj)}")

# print_shipment_route(scalable_model, title = "Optimal Shipment Plan", tol=1e-6)

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Jan 21 2024 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp69qi1jlm.pyomo.lp -stat=1 -solve -solu /tmp/tmp69qi1jlm.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 15 (0) rows, 50 (0) columns and 100 (0) elements
Statistics for presolved model


Problem has 15 rows, 50 columns (50 with objective) and 100 elements
Column breakdown:
50 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
10 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
5 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 15 (0) rows, 50 (0) columns and 100 (0) elements
Perturbing problem by 0.001% of 10 - largest n

Warehouse,Customer Region,Units Shipped
W1,R1,6.0
W1,R2,34.0
W1,R10,21.0
W2,R6,25.0
W2,R8,47.0
W3,R1,29.0
W3,R5,54.0
W4,R3,28.0
W4,R4,26.0
W4,R7,57.0


### Results Interpretation (Large-Scale Model)

This large-scale scenario demonstrates that the same optimization formulation
can be applied to transportation problems with increased size and complexity.
The model identifies a cost-minimizing shipment plan while satisfying all
operational constraints.

Key observations from the solution include:
- Shipment decisions reflect the objective of minimizing total transportation
  cost, with lower-cost routes preferred where feasible.
- Supply and demand constraints are satisfied for all warehouses and customer
  regions.
- The formulation remains structurally independent of problem size, allowing the
  same model to be applied across different transportation network scales.



## Key Takeaways

- The transportation problem can be formulated as a linear program, enabling
  cost-efficient logistics and distribution decisions.

- A single mathematical formulation applies seamlessly to both small reference scenario
  examples and large-scale, realistic transportation networks.

- Structured data generation enables reproducible optimization experiments and
  facilitates straightforward extension to both small and large-scale problem
  scenarios.


## Solver Integration & Comparison (CBC vs CPLEX CE)
To evaluate solver portability and demonstrate solver awareness, the transportation model is additionally solved using CPLEX (Community Edition) and compared against the existing CBC baseline.



In [8]:
#### Check the CPLEX availability

pyo.SolverFactory('cplex').available()

True

### Solver Comparison Results

In [9]:
### Solving with CPLEX (Community Edition)

model_cplex = build_transportation_model(warehouses, customer_regions, supply_capacity, demand, cost)
results_cplex = solve_transportation_problem(model_cplex, 'cplex')

cplex_optimal = pyo.value(model_cplex.obj)
print(f"CPLEX optimal cost: {cplex_optimal}")




Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer Community Edition 22.1.2.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2024.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmpc3s__4w6.cplex.log' open.
CPLEX> Problem '/tmp/tmpbvw0dgoz.pyomo.lp' read.
Read time = 0.02 sec. (0.00 ticks)
CPLEX> Problem name         : /tmp/tmpbvw0dgoz.pyomo.lp
Objective sense      : Minimize
Variables            :      50
Objective nonzeros   :      50
Linear constraints   :      15  [Less: 5,  Equal: 10]
  Nonzeros           :     100
  RHS nonzeros       :      15

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonzeros   : Min   : 3.000000         Max   : 10.00000       
Linear constraints   :
  Nonzeros           : Min   :

In [10]:
## Tabulating the comparison results

import pandas as pd

## to print in wide table format
solver_comparison = pd.DataFrame({
    "Solver": ["CBC", "CPLEX (CE)"],
    "Optimal Cost": [cbc_optimal, cplex_optimal]
})
solver_comparison.style.hide(axis="index").format({"Optimal Cost": "{:.2f}"})

# # to print in long format
# solver_comparison = pd.DataFrame({
#     "Metric": ["Optimal Cost"],
#     "CBC": [cbc_optimal],
#     "CPLEX (CE)": [cplex_optimal]
# })

# solver_comparison.style.hide(axis="index").format(
#     "{:.2f}",
#     subset=["CBC", "CPLEX (CE)"]
# )

Solver,Optimal Cost
CBC,1413.00
CPLEX (CE),1413.00



### Solver Comparison Insights

The CPLEX Community Edition solver returns the same optimal objective value as
the previously obtained CBC solution. This confirms that the transportation
model formulation is solver-independent and robust across different LP solvers.

This result demonstrates solver portability of the formulation, which is an
important requirement for real-world optimization pipelines where solver choice
may vary based on licensing, performance, or deployment constraints.

